## Training process for ECD507 Senior Capstone Project - F1Tenth ML Based Autonomous Race Car
### Contributors - Charles Hodgins, Rishabh Hegde, Dylan DiGiacomo, and Andrew Meccariello

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import ast

In [2]:
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.6


In [3]:
#index from around 100,000 for object avoidance

class DrivingDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_path = self.data.iloc[idx,1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (200, 66))  # Resize using OpenCV
        image = image / 255.0  # Normalize pixel values to [0,1]
        image = np.transpose(image, (2, 0, 1))  # Change shape to (C, H, W)
        image = torch.tensor(image, dtype=torch.float32)
        steering_angle = float(self.data.iloc[idx, 3])
        throttle = float(self.data.iloc[idx, 4])
        lidar = torch.tensor(ast.literal_eval(self.data.iloc[idx, 2]))
               
        return image, lidar, torch.tensor([steering_angle, throttle], dtype=torch.float32)


In [4]:
df = pd.read_csv("/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log3.csv")
df.columns = ["timestamp", "image_path","lidar_list","steering", "throttle", "rotational", "linear"]
df.to_csv("/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log3.csv", header=True, index=False)

In [ ]:
def filter_chunk(df):
    global first_chunk
    print(first_chunk)
    print(f"Initial size: {len(df)}")

    def is_valid_image(img_path):
        return os.path.exists(img_path) and cv2.imread(img_path) is not None

    
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]

    # before_throttle = len(df)
    # df = df[df["throttle"] != 0 ]
    # df = df[df['throttle'].apply(is_float)].copy()
    # df = df[df["throttle"] > 0.1]
    
    # print(f"Filtered throttle: {before_throttle - len(df)} rows removed")

    # before_lidar = len(df)
    df = df[df["lidar_list"] != ""]
    df = df[pd.notna(df["lidar_list"])]
    # print(f"Filtered lidar: {before_lidar - len(df)} rows removed")  
    df_filtered = df
    # before_images = len(df)
    # df["image_path"] = df["image_path"].str.strip()
    # df = df[df["image_path"] != ""]
    # df_filtered = df[df["image_path"].apply(is_valid_image)]
    # print(f"Filtered images: {before_images - len(df_filtered)} rows removed")

    # print(f"Final size: {len(df_filtered)}")
    
    print(len(df_filtered))
    
    if first_chunk:
        df_filtered.to_csv("/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log2.csv", mode='w', index=False, header=True)
        first_chunk = False
    else:
        df_filtered.to_csv("/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log2.csv", mode='a', index=False, header=False)


    # Save the cleaned dataset


    
first_chunk = True
index = 0
for chunk in pd.read_csv("/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log3.csv",chunksize = 10000):
    index+=1
    print(f'loaded chunk {index}')
    filter_chunk(chunk)
    # print(chunk.head())
    del chunk
    
print('done')

loaded chunk 1
True
Initial size: 10000


NameError: name 'df_filtered' is not defined

In [ ]:
dataset = DrivingDataset('/media/ecd507/JetsonOrinNano/home/ecd507/training/data/driving_log2.csv')
print(len(dataset))

dataloader = DataLoader(dataset, batch_size=32,
                        num_workers=4,
                        shuffle=True)

14449


In [ ]:
class LidarNet(nn.Module):
    def __init__(self):
        super(LidarNet, self).__init__()
        # expects input size of 721
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=24, kernel_size=10, stride=4) # output: (721-10)/4 + 1 = 178
        self.conv2 = nn.Conv1d(in_channels=24, out_channels=36, kernel_size=8, stride=4) # (178-8)/4 + 1 = 43
        self.conv3 = nn.Conv1d(in_channels=36, out_channels=48, kernel_size=4, stride=2) # (43-4)/2 + 1 = 20
        self.conv4 = nn.Conv1d(in_channels=48, out_channels=64, kernel_size=3, stride=1) # (20 -3)/1 + 1 = 1
        self.conv5 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=3, stride=1) # (18 - 3)/1 + 1 = 16

        self.fc1 = nn.Linear(1024, 512)  # (64 * 16) = 1024 -> 512
        self.fc2 = nn.Linear(512, 256) # 512 -> 256
        self.fc3 = nn.Linear(256, 100) # 256 -> 200
        self.fc4 = nn.Linear(100, 10) # 100 -> 10
        self.fc_out = nn.Linear(10, 2)  # 10 -> 2 Steering Angle & Speed

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))
        x = x.view(x.size(0), -1)  # Flatten to (batch, 1024)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc_out(x)  # No activation for raw output
        return x


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # change to cuda when it begins to work
print(device)
model = LidarNet().to(device)
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001, )
# Training loop
epochs = 20
for epoch in range(epochs):
    total_loss = 0
    batch = 0
    for images,lidar,targets in dataloader:
        lidar = lidar.unsqueeze(1)
        lidar, targets = lidar.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(lidar)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        batch+=1
        print(f'batch{batch}',end = '\r')
    
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader)}')

# Save model
torch.save(model.state_dict(), os.path.expanduser('~/models/lidar_model.pth'))
print("Model training complete and saved as lidar_model.pth")

cuda


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0xffff2d11e3b0>
Traceback (most recent call last):
  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0xffff2d11e3b0>
Traceback (most recent call last):
  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0xffff2d11e3b0>
Traceback (most recent call last):
  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
  

can only test a child process


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0xffff2d11e3b0>
Traceback (most recent call last):
  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0xffff2d11e3b0>
Traceback (most recent call last):
  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
    Exception ignored in: if w.is_alive():
<function _MultiProcessingDataLoaderIter.__del__ at 0xffff2d11e3b0>  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive

    assert self._parent_pid == os.getpid(), 'can only test a child process'Traceback (most recent call last):
  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__

    AssertionErrorself._shutdown_workers(): 
can only test a child process  File "/home/ecd507/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers

  

Exception ignored in: Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0xffffb5de4610>>
Traceback (most recent call last):


<bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0xffffb5de4610>>  File "/home/ecd507/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames


KeyboardInterrupt: 


Traceback (most recent call last):
  File "/home/ecd507/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
    def _clean_thread_parent_frames(KeyboardInterrupt
: 
KeyboardInterrupt: 


In [ ]:
torch.save(model.state_dict(), '/media/ecd507/JetsonOrinNano/home/ecd507/training/lidar_model.pth')
print("Model training complete and saved as lidar_model.pth")